# <font color="purple">Adversarial Training of Residual Network</font>

Definition of WideResNet and Architecture:

Residual Networks vs Wide Residual Networks

##  <center>The Result of Residual Network </center>

***Road Map***
* Data Preprocessing
* Model Cross Validation Results
* Evaluate the GridSearchCV Results
* Model Training and Learning Curves
* Model Adversarial Training Approach.


**Libraries**

In [2]:
import sys
sys.path.insert(1,'/home/sefika/AE_Parseval_Network/src/cleverhans/future/tf2/attacks')
import cleverhans


In [8]:
import tensorflow as tf
from cleverhans.future.tf2.attacks import fast_gradient_method
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from wresnet import WideResidualNetwork
from sklearn.model_selection import  KFold
import gzip
import pickle
from adversarial_training import AdversarialTraining
import numpy as np

import warnings
warnings.filterwarnings("ignore")

print("\nTensorflow Version: " + tf.__version__)



Tensorflow Version: 2.3.0


## <font color="green"> Data Preprocessing </font>


* Read Data from File

In [9]:
def read_data():
    with open("data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    return data
data = read_data()

* Call data preprocessing function

In [10]:
from preprocessing import preprocessing_data

In [11]:
X, y = preprocessing_data(data)
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size = 0.1)
x_train, X_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.1)

## <font color="green"> Utilize Functions </font>

* Flipping the image using data augmentation technique

In [111]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)

* Print the results of the experiments

In [13]:
def print_test(model,X_adv, X_test, y_test, epsilon):
    """
        returns the test results and show the SNR and evaluation results
    """
    loss, acc = model.evaluate(X_adv,y_test)
    print("epsilon: {} and test evaluation : {}, {}".format(epsilon,loss, acc))
    SNR = 20*np.log10(np.linalg.norm(X_test)/np.linalg.norm(X_test-X_adv))
    print("SNR: {}".format(SNR))
    return loss, acc

*  CleverHans Import and Functions Regarding Adversarial Exaples.

In [14]:
def get_adversarial_examples(pretrained_model, X_true, y_true, epsilon):
    """
        The attack requires the model to ouput the logits
        returns the adversarial example/s of a given image/s for epsilon value using
        fast gradient sign method
    """
    logits_model = tf.keras.Model(pretrained_model.input,pretrained_model.layers[-1].output)
    X_adv = []
    
    for i in range(len(X_true)):
        
        random_index = i
        
        original_image = X_true[random_index]
        original_image = tf.convert_to_tensor(original_image.reshape((1,32,32))) #The .reshape just gives it the proper form to input into the model, a batch of 1 a.k.a a tensor
        original_label = y_true[random_index]
        original_label = np.reshape(np.argmax(original_label), (1,)).astype('int64')
        
        adv_example_targeted_label = fast_gradient_method(logits_model, original_image, epsilon, np.inf,y=original_label, targeted=False)
        X_adv.append(np.array(adv_example_targeted_label).reshape(32,32,1))
        
    X_adv = np.array(X_adv)
    
    return X_adv

### <font color = "green">Some Parameters Regarding Adversarial Examples</font>

In [15]:
# predefined epsilon values
epsilon_list = [0.003,0.005,0.01,0.02]

### <font>General Visulazation Methods</font>

* Show accurancy and loss function of the results

In [203]:
import matplotlib.pyplot as plt

def show_graph(hist):
    """
        summarize history for accuracy
        plot the learning curves from history of the trained model
    """
    history = hist
     
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()


## <font color="purple"> 1.) Baseline of the Model</font>

* Explain the Baseline Model here

In [327]:
EPOCHS = 50
BS = 64
init = (32, 32,1)
sgd = SGD(lr=0.1, momentum=0.9)

conv2:channel:  -1
Wide Residual Network-16-1 created.
Finished compiling


In [324]:
# Define configuration parameters
import math
def step_decay(epoch):
    """[summary]

    Args:
        epoch (int): epoch number

    Returns:
        float: new learning rate
    """
    initial_lrate = 0.1
    factor = 0.1
    if epoch < 10:
      lrate = initial_lrate
    elif epoch < 20:
      lrate = initial_lrate*math.pow(factor, 1)
    elif epoch < 30:
      lrate = initial_lrate*math.pow(factor, 2)
    elif epoch < 40 :
      lrate = initial_lrate*math.pow(factor, 3)
    else:
      lrate = initial_lrate*math.pow(factor, 4)
    return lrate



lrate = LearningRateScheduler(step_decay)

In [320]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
callbacks_list = [lrate, es]
wresnet_ins = WideResidualNetwork(0.0001, init,0.9, nb_classes=4, N=2, k=1, dropout=0.0)

In [351]:
from training import train


In [ ]:
result_df = train(wresnet_ins, X_train, Y_train, X_test, y_test, EPOCHS, BS, sgd, generator, callbacks_list)
result_df["clean_mean"] = np.sum(result_df['acc_clean'])/10.0


In [353]:
result_df.head(1)

,loss_clean,acc_clean,clean_mean
0,0.80251,0.726003,0.722688


## <font color="purple">2.) Adversarial Training on Baseline Model</font>

In [ ]:
adversarial_training =  AdversarialTraining()
result_adv_df = adversarial_training.train(wresnet_ins, X_train, Y_train, X_test, y_test, EPOCHS, BS, epsilon_list, sgd, callbacks_list)

### <font color="purple"> Results of Adversarial Training</font>

In [343]:
result_adv_df["clean_mean"] = np.sum(result_adv_df['acc_clean'])/10.0
result_adv_df["0.003_mean"] = np.sum(result_adv_df['acc1'])/10.0
result_adv_df["0.005_mean"] = np.sum(result_adv_df['acc2'])/10.0
result_adv_df["0.02_mean"] = np.sum(result_adv_df['acc3'])/10.0
result_adv_df["0.01_mean"] = np.sum(result_adv_df['acc4'])/10.0

In [344]:
column = ["clean_mean","0.003_mean","0.005_mean","0.02_mean","0.01_mean"]
result_adv_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.722339,0.676789,0.644852,0.573822,0.442932


# <font color="blue">Compare Non-Adversarial Training with Adversarial Training</font>

In [354]:
result_adv_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.722339,0.676789,0.644852,0.573822,0.442932


In [355]:
result_df.head(1)

,loss_clean,acc_clean,clean_mean
0,0.80251,0.726003,0.722688


# <font color="blue">Conclusion</font>